<a href="https://colab.research.google.com/github/NathanAugusth/Algoritmos-e-Complexidade/blob/main/Extra%C3%A7%C3%A3o_de_Noticias_com_BeautifulSoup_%2B_Requests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import requests
from bs4 import BeautifulSoup
import csv
import os

def scrape_jc_uol():
    url = 'https://jc.uol.com.br/'
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        # Busca por elementos que podem conter manchetes: h1, h2, h3, a com classes comuns
        headlines = []
        selectors = ['h1 a', 'h2 a', 'h3 a', 'article h2', 'article h3', '.news-title a', '.title a', 'a[href*="jc.uol.com.br"]']

        for selector in selectors:
            elements = soup.select(selector)
            for elem in elements:
                if elem.get('href') and elem.text.strip():
                    link = elem.get('href')
                    if not link.startswith('http'):
                        link = 'https://jc.uol.com.br' + link if link.startswith('/') else 'https://jc.uol.com.br/' + link
                    title = elem.text.strip()
                    if title and len(title) > 10:  # Filtra títulos muito curtos
                        # Procura por data próxima ao elemento (opcional)
                        date = None
                        parent = elem.parent
                        if parent:
                            date_elem = parent.find(['time', '.date', '.data'])
                            if date_elem:
                                date = date_elem.text.strip()

                        headlines.append({
                            'titulo': title,
                            'link': link,
                            'data': date or ''
                        })
                        if len(headlines) >= 20:  # Limita a 20 manchetes para evitar duplicatas excessivas
                            break
            if len(headlines) >= 20:
                break

        # Remove duplicatas baseadas no link
        seen_links = set()
        unique_headlines = []
        for h in headlines:
            if h['link'] not in seen_links:
                seen_links.add(h['link'])
                unique_headlines.append(h)

        return unique_headlines

    except Exception as e:
        print(f"Erro ao acessar a página: {e}")
        return []

def display_headlines(headlines):
    if not headlines:
        print("Nenhuma manchete encontrada.")
        return

    print("\nManchetes Principais do JC UOL:\n")
    print("-" * 80)
    for i, news in enumerate(headlines, 1):
        print(f"{i}. Título: {news['titulo']}")
        print(f"   Link: {news['link']}")
        if news['data']:
            print(f"   Data: {news['data']}")
        print()

def save_to_csv(headlines, filename='manchetes_jc_uol.csv'):
    if not headlines:
        print("Nenhum dado para salvar.")
        return

    fieldnames = ['titulo', 'link', 'data']
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(headlines)
    print(f"Dados salvos em {filename}")

if __name__ == "__main__":
    headlines = scrape_jc_uol()
    display_headlines(headlines)
    save_to_csv(headlines)


Manchetes Principais do JC UOL:

--------------------------------------------------------------------------------
1. Título: Recife inicia construção de habitacional no bairro de Boa Viagem
   Link: https://jc.uol.com.br/colunas/metro-quadrado/2025/09/16/recife-inicia-construcao-de-habitacional-no-bairro-de-boa-viagem.html

2. Título: BLOG DO TORCEDOR
   Link: https://blogdotorcedor.com.br/

3. Título: Edição do dia
   Link: https://impresso.jc.uol.com.br/index.php?id=/banca.php&utm_source=site_jc&utm_medium=imagem_capa&utm_campaign=chama_flip

4. Título: JORNAL DIGITAL
   Link: https://digital.jc.uol.com.br/

5. Título: Blog do Torcedor
   Link: https://jc.uol.com.br/blog-do-torcedor/

6. Título: Recall de Marcas
   Link: https://jc.uol.com.br/recall-de-marcas

7. Título: SANEAMENTO
Recife é uma das seis capitais brasileiras que não evoluíram na coleta de esgoto
De acordo com um estudo do Instituto Trata Brasil, entre os anos de 2019 e 2023, capital pernambucana regrediu no abastecim